<a href="https://colab.research.google.com/github/RakerZh/cse404/blob/main/cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/dataset
!ls


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')


In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
import numpy as np

def preprocess_image(img_path):
    img = load_img(img_path, target_size=(400, 300))
    x = img_to_array(img)
    x = x.astype('float32') / 255.0
    return x

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.applications import ResNet50

# Load data from CSV file
import pandas as pd
data = pd.read_csv('./data/english.csv')

images = []
for image_path in data['image']:
  print(image_path)
  images.append(preprocess_image("./data/"+image_path)) 



In [ ]:
from sklearn.preprocessing import LabelEncoder
# Encode labels as numerical values
label_encoder = LabelEncoder()
data['label'] = label_encoder.fit_transform(data['label'])

# Preprocess images
images = [preprocess_image("./data/"+image_path) for image_path in data['image']]
X = np.array(images)
y = to_categorical(data['label'], num_classes=62)  # specify the number of classes

# Define model
model = Sequential()
model.add(ResNet50(include_top=False, pooling='avg', weights='imagenet'))
model.add(Dense(62, activation='softmax'))  # set the number of units to 62

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
history = model.fit(X, y, epochs=10, batch_size=32, validation_split=0.2)



In [ ]:
import matplotlib.pyplot as plt
print(data['label'])

In [ ]:
# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()